# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [8]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [13]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [14]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather information on DeepLearningAI to create a comprehensive report for our sales team.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: Title: About - DeepLearning.AI
Link: https://www.deeplearning.ai/about/
Snippet: DeepLearning.AI is an education technology company that is empowering the global workforce to build an AI-powered 

2025-01-10 22:15:46,909 - 139651158911872 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--f28e5ca6e13b3612a49d71bceda1298d6382619b25dff0def913f3fd66aa7528


Thought: 
Since I couldn't access the DeepLearningAI Company Profile file, I will need to rely on the information gathered from the internet search results to create a comprehensive report.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile Andrew Ng"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Action: Search the internet
Action Input: {"search_query": "DeepLearningAI Andrew Ng company profile"} 


Search results: Title: Andrew Ng - Founder - DeepLearning.AI - LinkedIn
Link: https://www.linkedin.com/in/andrewyng
Snippet: Founder of DeepLearning.AI; Managing General Partner of AI Fund; Exec Chairman of Landing AI · Personal home page: http://andrewng.org · Experience: ...
---
Title: Deeplearning.ai - Crunchbase Company Profile & Funding
Link: https://www.crunchbase.com/organization/deeplearning-ai
Snippet: Deeplearning.ai is company that explores the frontier of AI. The company is fo

Thought: 
I need to incorporate DeepLearningAI's recent product launch, the "Generative AI for Everyone" course on Coursera, into my personalized outreach campaign for Andrew Ng.

Action: Search the internet
Action Input: {"search_query": "Generative AI for Everyone DeepLearningAI Coursera"} 


Search results: Title: Generative AI for Everyone - Coursera
Link: https://www.coursera.org/learn/generative-ai-for-everyone
Snippet: You'll get insights into what generative AI can do, its potential, and its limitations. You'll delve into real-world applications and learn common use cases.
---
Title: Generative AI for Everyone - DeepLearning.AI
Link: https://www.deeplearning.ai/courses/generative-ai-for-everyone/
Snippet: Instructed by AI pioneer Andrew Ng, Generative AI for Everyone offers his unique perspective on empowering you and your work with generative AI.
---
Title: Best Generative AI (GenAI) Courses Online with Certificates [2024]
Link: https://www.coursera.org/courses?query=generativ

- Display the final result as Markdown.

In [15]:
from IPython.display import Markdown
Markdown(result)

Email Draft 1: Subject: Enhancing Your AI Education Platform with DeepLearningAI Solutions 

Dear Andrew Ng,

I hope this email finds you well. I recently came across the exciting news about the launch of the "Generative AI for Everyone" course on Coursera by DeepLearningAI. As a global leader in AI education, your commitment to empowering learners worldwide is truly inspiring.

At CrewAI, we share your vision of democratizing deep learning and advancing the field of artificial intelligence. Our solutions are designed to enhance online learning platforms like yours, providing valuable insights and tools to drive innovation in the AI education sector.

I believe that by collaborating with DeepLearningAI, we can create a mutually beneficial partnership that fosters growth and success for both our companies. I would love the opportunity to discuss how our solutions can support your goals and contribute to the continued success of DeepLearningAI.

Looking forward to the possibility of working together.

Warm regards,
[Your Name]
Lead Sales Representative
CrewAI

Email Draft 2: Subject: Driving Innovation in AI Education with CrewAI 

Dear Andrew Ng,

I trust this email finds you well. I wanted to reach out and congratulate you on the recent launch of the "Generative AI for Everyone" course on Coursera. Your dedication to providing world-class AI education accessible to people around the globe is truly commendable.

At CrewAI, we are passionate about driving innovation in the AI education sector. Our solutions are tailored to enhance online learning platforms like DeepLearningAI, offering valuable tools and insights to support your mission of empowering the global workforce to build an AI-powered future.

I believe that by partnering with DeepLearningAI, we can create a synergy that propels both our companies forward. I am excited about the possibility of collaborating with you to further advance the field of artificial intelligence and make a meaningful impact on the global workforce.

Thank you for your time, and I look forward to the opportunity to discuss how our solutions can benefit DeepLearningAI.

Best regards,
[Your Name]
Lead Sales Representative
CrewAI